# Exploration

A notebook used to explore FRED data

## Setup    

In [1]:
# Load environment variables using dotenv

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Setup autoreload of modules
%load_ext autoreload
%autoreload 2


In [3]:
from update_path import ensure_src_in_path

ensure_src_in_path()

/Users/coryhuebner/workspace/fed/fred-data/src already in path


In [4]:
# Get the FRED API client

from fred_data import api

%autoreload 2

fred_client = api.FredApiClient()

## Data Exploration

In [5]:
# Fetch all current releases
from fred_data.releases import get_releases

releases = get_releases(fred_client)
releases_df = releases.releases

# Get an idea of the amount/types of data stored in releases
print(f"Release summary")
print(f"Count: {releases.count}")
print(f"Realtime range: {releases.realtime_start} to {releases.realtime_end}, Duration: {releases.realtime_end - releases.realtime_start}")
releases.releases.describe()

Release summary
Count: 315
Realtime range: 2025-03-11 to 2025-03-11, Duration: 0:00:00


statistic,id,realtime_start,realtime_end,name,press_release,link,notes
str,f64,str,str,str,f64,str,str
"""count""",315.0,"""315""","""315""","""315""",315.0,"""276""","""123"""
"""null_count""",0.0,"""0""","""0""","""0""",0.0,"""39""","""192"""
"""mean""",288.031746,"""2025-03-11 00:00:00""","""2025-03-11 00:00:00""",null,0.542857,null,null
"""std""",152.199623,null,null,null,null,null,null
"""min""",9.0,"""2025-03-11""","""2025-03-11""","""A Millennium of Macroeconomic …",0.0,"""http://data.imf.org/?sk=3E40CD…","""AMERIBOR® (American Interbank …"
"""25%""",172.0,"""2025-03-11""","""2025-03-11""",null,null,null,null
"""50%""",301.0,"""2025-03-11""","""2025-03-11""",null,null,null,null
"""75%""",406.0,"""2025-03-11""","""2025-03-11""",null,null,null,null
"""max""",769.0,"""2025-03-11""","""2025-03-11""","""Zillow Home Value Index (ZHVI)""",1.0,"""https://www2.optimalblue.com/o…","""With the issue dated January 8…"


In [6]:
# Fetch older and current releases

from datetime import date, timedelta
from fred_data.releases import get_releases

archived_releases = get_releases(fred_client, realtime_start=date.today() - timedelta(days=60), realtime_end=date.today())
archived_releases_df = archived_releases.releases

# Get an idea of the amount/types of data stored in releases
print(f"Archived Release summary")
print(f"Count: {archived_releases.count}")
print(f"Realtime range: {archived_releases.realtime_start} to {archived_releases.realtime_end}, Duration: {archived_releases.realtime_end - archived_releases.realtime_start}")
archived_releases.releases.describe()

Archived Release summary
Count: 309
Realtime range: 2025-01-10 to 2025-03-11, Duration: 60 days, 0:00:00


statistic,id,realtime_start,realtime_end,name,press_release,link,notes
str,f64,str,str,str,f64,str,str
"""count""",309.0,"""309""","""309""","""309""",309.0,"""270""","""121"""
"""null_count""",0.0,"""0""","""0""","""0""",0.0,"""39""","""188"""
"""mean""",287.640777,"""2025-01-10 00:00:00""","""2025-03-11 00:00:00""",null,0.540453,null,null
"""std""",152.758267,null,null,null,null,null,null
"""min""",9.0,"""2025-01-10""","""2025-03-11""","""A Millennium of Macroeconomic …",0.0,"""http://data.imf.org/?sk=3E40CD…","""AMERIBOR® (American Interbank …"
"""25%""",170.0,"""2025-01-10""","""2025-03-11""",null,null,null,null
"""50%""",302.0,"""2025-01-10""","""2025-03-11""",null,null,null,null
"""75%""",403.0,"""2025-01-10""","""2025-03-11""",null,null,null,null
"""max""",769.0,"""2025-01-10""","""2025-03-11""","""Zillow Home Value Index (ZHVI)""",1.0,"""https://www2.optimalblue.com/o…","""With the issue dated January 8…"


In [ ]:
# Get release dates using default time range
from fred_data.releases import get_release_dates

release_dates = get_release_dates(fred_client)
release_dates_df = release_dates.release_dates

print(f"Release Dates Summary")
print(f"Count: {release_dates.count}")
print(f"Realtime range: {release_dates.realtime_start} to {release_dates.realtime_end}, Duration: {release_dates.realtime_end - release_dates.realtime_start}")
release_dates_df.describe()

TypeError: get_release_dates() missing 2 required positional arguments: 'realtime_start' and 'realtime_end'

In [18]:
# Get release dates from the past into the future
from fred_data.releases import get_release_dates

release_dates_over_time = get_release_dates(
    fred_client,
    realtime_start=date.today() - timedelta(days=30),
    realtime_end=date.today() + timedelta(days=10),
    include_release_dates_with_no_data=True
)
release_dates_over_time_df = release_dates_over_time.release_dates

print(f"Release Dates Summary")
print(f"Count: {release_dates_over_time.count}")
print(f"Realtime range: {release_dates_over_time.realtime_start} to {release_dates_over_time.realtime_end}, Duration: {release_dates_over_time.realtime_end - release_dates_over_time.realtime_start}")
release_dates_over_time_df.describe()

KeyboardInterrupt: 